In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.gridspec import GridSpec
import numpy as np
import filehandling
from ball_bearing_duty import duty_to_dimensionless_acceleration
import os
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [19]:
direc = "/media/data/Data/BallBearing/HIPS/IslandExperiments/New_2,25mm_experiments/85%_15s"
files = filehandling.get_directory_filenames(f"{direc}/*.hdf5")

In [20]:
duties = []
acc = []
crystal_0 = []
crystal_pi = []
for file in files:
    data = pd.read_hdf(file)
    duty = int(file[-8:-5])
    ordered = np.abs(data.hexatic_order)>0.8
    angles = np.angle(data.hexatic_order.values[ordered])

    duties.append(duty)
    acc.append(duty_to_dimensionless_acceleration(duty))
    count_0 = np.count_nonzero((-np.pi/3 < angles)&(angles < np.pi/3))
    count_pi = np.count_nonzero((angles<-2*np.pi/3)|(angles>2*np.pi/3))
    crystal_0.append(count_0/len(data)*100)
    crystal_pi.append(count_pi/len(data)*100)
crystal_0 = np.array(crystal_0)
crystal_pi = np.array(crystal_pi)

In [21]:
plot_duty = False


matplotlib.rcParams.update({'font.size': 22})
fig = plt.figure(figsize=(13.5, 10))
gridspec = GridSpec(2, 3, height_ratios=[1, 1])
cmap = matplotlib.cm.hsv
norm = matplotlib.colors.Normalize(vmin=-np.pi, vmax=np.pi)

x = duties if plot_duty else acc

ax_plot = fig.add_subplot(gridspec[0, :])
ax_plot.plot(x, crystal_0, 'b', label='0')
ax_plot.plot(x, crystal_pi, 'r', label='$\pi$')
# ax_plot.plot(x, crystal_0+crystal_pi, 'k', label='Total')
ax_plot.set_xlabel('$\Gamma$')
ax_plot.set_ylabel('%')
ax_plot.legend()
# ax_plot.set_ylim([0, 30])

duties_to_plot = 480, 486, 510
# duties_to_plot = 450, 480, 495 # 5s
# duties_to_plot = 470, 490, 500 # 30
duties_to_plot = 480, 490, 510 # 300
for i, duty in enumerate(duties_to_plot):
    if plot_duty:
        ax_plot.axvline(duty, ls='--')
    else:
        ax_plot.axvline(duty_to_dimensionless_acceleration(duty), ls='--')
    ax = fig.add_subplot(gridspec[1, i])
    ax.set_aspect('equal')
    ax.xaxis.set_visible(False)
    ax.yaxis.set_ticks([])
    ax.yaxis.set_ticklabels([])
    fname = f"{direc}/{duty}.hdf5"
    data = pd.read_hdf(fname)
    frame = data.loc[0].copy()
    frame['angle_abs'] = np.abs(np.angle(frame.hexatic_order))
    frame['hex_abs'] = np.abs(frame.hexatic_order)
    disorderd = frame.loc[frame.hex_abs<0.8]
    ax.scatter(disorderd.x, disorderd.y, c='k', s=1)

    ordered_0 = frame.loc[(frame.hex_abs>0.8)*(frame.angle_abs<np.pi/3)]
    plt.scatter(ordered_0.x, ordered_0.y, c='b', s=1)

    ordered_pi = frame.loc[(frame.hex_abs>0.8)*(frame.angle_abs>2*np.pi/3)]
    plt.scatter(ordered_pi.x, ordered_pi.y, c='r', s=1)

plt.savefig(f"{direc}/overview.png", dpi=600)